In [ ]:
from scipy.io import wavfile as wv
import scipy.signal as ss
from scipy.stats import entropy
import numpy as np
import os
import librosa
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import FastICA
from sklearn.cluster import KMeans
from sklearn.svm import SVC
import sklearn.naive_bayes as sknb
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [ ]:
td = pd.read_csv('./train.csv')
y = td['Genre'].values

In [ ]:
# Directory to my training set should change
directory_test = "/Users/noahvilla/Downloads/Dataset/test"

audio_data_list_test = []
sampling_rate_list_test = []

# Iterate over training data
for filename in os.listdir(directory_test):
    # Loads all the .wav files in the traning set
    if filename.endswith(".wav"):
        file_path = os.path.join(directory_test, filename)
        audio_data, sampling_rate = librosa.load(file_path, sr=None)
        
        # Append the audio data and sampling rate to the lists
        audio_data_list_test.append(audio_data)
        sampling_rate_list_test.append(sampling_rate)

# Now we can use the audio data and sampling rate data

In [ ]:
# Directory to my training set should change
directory = "/Users/noahvilla/Downloads/Dataset/train"

audio_data_list = []
sampling_rate_list = []

# Iterate over training data
for filename in os.listdir(directory):
    # Loads all the .wav files in the traning set
    if filename.endswith(".wav"):
        file_path = os.path.join(directory, filename)
        audio_data, sampling_rate = librosa.load(file_path, sr=None)
        
        # Append the audio data and sampling rate to the lists
        audio_data_list.append(audio_data)
        sampling_rate_list.append(sampling_rate)

# Now we can use the audio data and sampling rate data

In [ ]:
#Gathers the frequency information of a song
fft_list = []
for song in audio_data_list:
    fft_list.append(np.fft.fft(song))
    
fft_list_test = []
for song in audio_data_list_test:
    fft_list_test.append(np.fft.fft(song))

In [ ]:
# Computes energy of each song
energy_list = []
for song in audio_data_list:
    energy_list.append(np.sum(np.square(song)))
    
energy_list_test = []
for song in audio_data_list_test:
    energy_list_test.append(np.sum(np.square(song)))

In [ ]:
# Finds the variance and mean of each song
variance_list = []
expectation_list = []
for song in audio_data_list:
    variance_list.append(np.var(song))
    expectation_list.append(np.mean(song))
    
variance_list_test = []
expectation_list_test = []
for song in audio_data_list_test:
    variance_list_test.append(np.var(song))
    expectation_list_test.append(np.mean(song))

In [ ]:
tempo_list = []
for i in range(0,len(audio_data_list)):
    tempo, _ = librosa.beat.beat_track(y=audio_data_list[i], sr=sampling_rate_list[i])
    tempo_list.append(tempo)

tempo_list_test = []
for i in range(0,len(audio_data_list_test)):
    tempo, _ = librosa.beat.beat_track(y=audio_data_list_test[i], sr=sampling_rate_list_test[i])
    tempo_list_test.append(tempo)

In [ ]:
zero_cross_rate_list = []
for song in audio_data_list:
    zero_cross_rate_list.append(np.median(librosa.feature.zero_crossing_rate(song)))

zero_cross_rate_list_test = []
for song in audio_data_list_test:
    zero_cross_rate_list_test.append(np.median(librosa.feature.zero_crossing_rate(song)))

In [ ]:
fft_entrp_list = []
for fft in fft_list:
    fft_entrp_list.append(entropy(np.absolute(fft)))

fft_entrp_list_test = []
for fft in fft_list_test:
    fft_entrp_list_test.append(entropy(np.absolute(fft)))

In [ ]:
mfcc_list = []
for song in audio_data_list:
    mfcc_list.append(librosa.feature.mfcc(y = song, n_mfcc = 15))

mfcc_list_test = []
for song in audio_data_list_test:
    mfcc_list_test.append(librosa.feature.mfcc(y = song, n_mfcc = 15))

In [ ]:
condensed_mfcc = []
for mfcc in mfcc_list:
    condensed_mfcc.append(np.average(mfcc))

condensed_mfcc_test = []
for mfcc in mfcc_list_test:
    condensed_mfcc_test.append(np.average(mfcc))

In [ ]:
var_mfcc = []
for mfcc in mfcc_list:
    var_mfcc.append(np.var(mfcc))

var_mfcc_test = []
for mfcc in mfcc_list_test:
    var_mfcc_test.append(np.var(mfcc))

In [ ]:
spectral_centroid_list = []
spectral_rolloff_list = []
spectral_bandwidth_list = []
spectral_contrast_list = []
spectral_flatness_list = []
onset_list = []
beat_list = []
beat_eng_list = []
chrom_list = []
for song in audio_data_list:
    spectral_centroid_list.append(librosa.feature.spectral_centroid(y = song))
    spectral_rolloff_list.append(librosa.feature.spectral_rolloff(y = song))
    spectral_flatness_list.append(librosa.feature.spectral_flatness(y = song))
    onset = librosa.onset.onset_strength(y = song)
    onset_list.append(onset)
    _, beat = librosa.beat.beat_track(y = song)
    beat_list.append(beat)
    beat_eng_list.append(onset[beat])
    chrom_list.append(librosa.feature.chroma_stft(y = song))
    spectral_bandwidth_list.append(librosa.feature.spectral_bandwidth(y = song))
    spectral_contrast_list.append(librosa.feature.spectral_contrast(y = song))

In [ ]:
spectral_centroid_list_test = []
spectral_rolloff_list_test = []
spectral_bandwidth_list_test = []
spectral_contrast_list_test = []
spectral_flatness_list_test = []
onset_list_test = []
beat_list_test = []
beat_eng_list_test = []
chrom_list_test = []
for song in audio_data_list_test:
    spectral_centroid_list_test.append(librosa.feature.spectral_centroid(y = song))
    spectral_rolloff_list_test.append(librosa.feature.spectral_rolloff(y = song))
    spectral_flatness_list_test.append(librosa.feature.spectral_flatness(y = song))
    onset = librosa.onset.onset_strength(y = song)
    onset_list_test.append(onset)
    _, beat = librosa.beat.beat_track(y = song)
    beat_list_test.append(beat)
    beat_eng_list_test.append(onset[beat])
    chrom_list_test.append(librosa.feature.chroma_stft(y = song))
    spectral_bandwidth_list_test.append(librosa.feature.spectral_bandwidth(y = song))
    spectral_contrast_list_test.append(librosa.feature.spectral_contrast(y = song))

In [ ]:
def getFeat(songs):
    expectation = []
    var = []
    entrp = []
    for song in songs:
        expectation.append(np.average(np.absolute(song)))
        var.append(np.var(song))
        entrp.append(entropy(song))
    return expectation, var, entrp

In [ ]:
spectral_centroid_exp_list, spectral_centroid_var_list, spectral_centroid_ent_list = getFeat(spectral_centroid_list)
spectral_rolloff_exp_list, spectral_rolloff_var_list, spectral_rolloff_ent_list = getFeat(spectral_rolloff_list)
spectral_bandwidth_exp_list, spectral_bandwidth_var_list, spectral_bandwidth_ent_list = getFeat(spectral_bandwidth_list)
spectral_flatness_exp_list, spectral_flatness_var_list, spectral_flatness_ent_list = getFeat(spectral_flatness_list)
spectral_contrast_exp_list, spectral_contrast_var_list, spectral_contrast_ent_list = getFeat(spectral_contrast_list)
chrom_exp_list, chrom_var_list, chrom_ent_list = getFeat(chrom_list)
beat_eng_exp_list, beat_eng_var_list, beat_eng_ent_list = getFeat(beat_eng_list)
onset_exp_list, onset_var_list, onset_ent_list = getFeat(onset_list)

In [ ]:
spectral_centroid_exp_list_test, spectral_centroid_var_list_test, spectral_centroid_ent_list_test = getFeat(spectral_centroid_list_test)
spectral_rolloff_exp_list_test, spectral_rolloff_var_list_test, spectral_rolloff_ent_list_test = getFeat(spectral_rolloff_list_test)
spectral_bandwidth_exp_list_test, spectral_bandwidth_var_list_test, spectral_bandwidth_ent_list_test = getFeat(spectral_bandwidth_list_test)
spectral_flatness_exp_list_test, spectral_flatness_var_list_test, spectral_flatness_ent_list_test = getFeat(spectral_flatness_list_test)
spectral_contrast_exp_list_test, spectral_contrast_var_list_test, spectral_contrast_ent_list_test = getFeat(spectral_contrast_list_test)
chrom_exp_list_test, chrom_var_list_test, chrom_ent_list_test = getFeat(chrom_list_test)
beat_eng_exp_list_test, beat_eng_var_list_test, beat_eng_ent_list_test = getFeat(beat_eng_list_test)
onset_exp_list_test, onset_var_list_test, onset_ent_list_test = getFeat(onset_list_test)

In [ ]:
spectral_contrast_ent_exp_list = []
spectral_contrast_ent_var_list = []
for item in spectral_contrast_ent_list:
    spectral_contrast_ent_exp_list.append(np.average(item))
    spectral_contrast_ent_var_list.append(np.var(item))

chrom_ent_exp_list = []
chrom_ent_var_list = []
for item in chrom_ent_list:
    chrom_ent_exp_list.append(np.average(item))
    chrom_ent_var_list.append(np.var(item))

go_through = (energy_list, variance_list, expectation_list, tempo_list, zero_cross_rate_list, 
              fft_entrp_list, condensed_mfcc, var_mfcc, fft_mean_list, fft_var_list, 
              spectral_centroid_exp_list, spectral_centroid_var_list, spectral_rolloff_exp_list, spectral_rolloff_var_list, spectral_bandwidth_exp_list, 
              spectral_bandwidth_var_list, spectral_flatness_exp_list, spectral_flatness_var_list, spectral_contrast_exp_list, spectral_contrast_var_list, 
              spectral_contrast_ent_exp_list, spectral_contrast_ent_var_list, chrom_exp_list, chrom_var_list, chrom_ent_exp_list, 
              chrom_ent_var_list, beat_eng_exp_list, beat_eng_var_list, beat_eng_ent_list, onset_exp_list, 
              onset_var_list, onset_ent_list)

def replace_nan_with_zero(array):
    array[np.isnan(array)] = 0
    return array

# Apply the function to each array in the tuple
go_through_cleaned = tuple(replace_nan_with_zero(np.array(arr)) for arr in go_through)

spectral_contrast_ent_exp_list_test = []
spectral_contrast_ent_var_list_test = []
for item in spectral_contrast_ent_list_test:
    spectral_contrast_ent_exp_list_test.append(np.average(item))
    spectral_contrast_ent_var_list_test.append(np.var(item))

chrom_ent_exp_list_test = []
chrom_ent_var_list_test = []
for item in chrom_ent_list_test:
    chrom_ent_exp_list_test.append(np.average(item))
    chrom_ent_var_list_test.append(np.var(item))

go_through_test = (energy_list_test, variance_list_test, expectation_list_test, tempo_list_test, zero_cross_rate_list_test, 
              fft_entrp_list_test, condensed_mfcc_test, var_mfcc_test, fft_mean_list_test, fft_var_list_test, 
              spectral_centroid_exp_list_test, spectral_centroid_var_list_test, spectral_rolloff_exp_list_test, spectral_rolloff_var_list_test, spectral_bandwidth_exp_list_test, 
              spectral_bandwidth_var_list_test, spectral_flatness_exp_list_test, spectral_flatness_var_list_test, spectral_contrast_exp_list_test, spectral_contrast_var_list_test, 
              spectral_contrast_ent_exp_list_test, spectral_contrast_ent_var_list_test, chrom_exp_list_test, chrom_var_list_test, chrom_ent_exp_list_test, 
              chrom_ent_var_list_test, beat_eng_exp_list_test, beat_eng_var_list_test, beat_eng_ent_list_test, onset_exp_list_test, 
              onset_var_list_test, onset_ent_list_test)

go_through_cleaned_test = tuple(replace_nan_with_zero(np.array(arr)) for arr in go_through_test)

In [ ]:
def conv_compare(song, data):
    """
    Sees how similar the fft of the songs are, higher number means more overlap
    song is one song and data is all the songs to compare song to
    can be used for time or for freq domain info
    """
    flipped_song = np.flip(song)
    convolution_sim_list = []
    for audio in data:
        sum = 0
        for i in range(0,len(audio)):
            sum += audio[i]*flipped_song[i]
        convolution_sim_list.append(sum)

In [ ]:
# Step 1: Prepare the Feature Matrix
feature_matrix = np.column_stack(go_through_cleaned)
feature_matrix_test = np.column_stack(go_through_cleaned_test)

# Step 2: Normalize the Feature Matrix
scaler = StandardScaler()
normalized_features = scaler.fit_transform(feature_matrix)
normalized_features_test = scaler.fit_transform(feature_matrix_test)

# Step 3: Apply ICA
ica = FastICA(n_components=10, random_state=42)
independent_components = ica.fit_transform(normalized_features)
independent_components_test = ica.fit_transform(normalized_features_test)

# Step 4: Clustering and Classifying
clf = SVC(kernel='linear')
clf.fit(independent_components, y)

# Map cluster labels to genres
predicted_genres = clf.predict(independent_components_test)

In [ ]:
nb_model = sknb.GaussianNB()
nb_model.fit(normalized_features, y)
predicted_genres = nb_model.predict(normalized_features_test)

In [ ]:
predictions = []
for filename, genre in zip(os.listdir(directory_test), predicted_genres):
    predictions.append((filename,genre))

In [ ]:
# Sort the list nurmerically since the dict is random
sorted_data = sorted(predictions, key=lambda x: x[0][5:8])

In [ ]:
# Puts it into a .csv file (finally!!!!)
df = pd.DataFrame(sorted_data, columns=['ID', 'genre'])
df.to_csv('/Users/noahvilla/Downloads/Dataset/test.csv', index=False)